In [132]:
import pickle
import pandas as pd

import sys
sys.path.append("..")
from src.support_prep import *

In [133]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [134]:
with open('../models/target.pkl', 'rb') as file:
    target = pickle.load(file)
with open('../models/onehot.pkl', 'rb') as file:
    onehot = pickle.load(file)
# Scaling
with open('../models/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
# Model
with open('../models/model_resampled.pkl', 'rb') as file:
    model = pickle.load(file)

In [135]:
cat_diff  = ['BusinessTravel',
            'Department',
            'EducationField',
            'JobRole',
            'MaritalStatus',
            'EnvironmentSatisfaction',
            'JobSatisfaction',
            'WorkLifeBalance',
            'JobInvolvement']

cat_no_diff = ['Education', 'Gender', 'JobLevel', 'StockOptionLevel','PerformanceRating']

In [136]:
test_series = pd.read_pickle("../datos/clean.pkl").drop(columns = ["Attrition", "EmployeeID"])


In [151]:
import requests

response = requests.post(url = "http://127.0.0.1:5000/predict", json = test_series.loc[2].to_json())
print(response)

<Response [500]>


In [139]:
for i in test_series.columns:
    print(i, test_series[i].dtypes, ":", test_series[i].unique())

Age int64 : [51 31 32 38 46 28 29 25 45 36 55 47 37 21 35 26 50 53 42 44 49 18 41 39
 58 33 43 52 27 30 54 40 23 48 57 34 24 22 56 60 19 20 59]
BusinessTravel object : ['Travel_Rarely' 'Travel_Frequently' 'Non-Travel']
Department object : ['Sales' 'Research & Development' 'Human Resources']
DistanceFromHome int64 : [ 6 10 17  2  8 11 18  1  7 28 14  3  4 16  9  5 20 29 15 13 24 19 22 25
 21 26 27 12 23]
Education object : ['2' '1' '4' '5' '3']
EducationField object : ['Life Sciences' 'Other' 'Medical' 'Marketing' 'Technical Degree'
 'Human Resources']
Gender object : ['Female' 'Male']
JobLevel object : ['1' '4' '3' '2' '5']
JobRole object : ['Healthcare Representative' 'Research Scientist' 'Sales Executive'
 'Human Resources' 'Research Director' 'Laboratory Technician'
 'Manufacturing Director' 'Sales Representative' 'Manager']
MaritalStatus object : ['Married' 'Single' 'Divorced']
MonthlyIncome int64 : [131160  41890 193280 ...  37020  23980  54680]
NumCompaniesWorked int32 : [1 0 3 4

In [140]:
res_target = target.transform(test_series)
res_target

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,51,0.149569,0.150224,6,2,0.166667,Female,1,0.145038,0.124814,...,1,6,1,0,0,0.137778,0.114119,0.167812,0.153226,3
1,31,0.249097,0.157128,10,1,0.166667,Female,1,0.181507,0.255319,...,6,3,5,1,4,0.137778,0.164286,0.178414,0.160000,4
2,32,0.249097,0.157128,17,4,0.121951,Male,4,0.168712,0.124814,...,5,2,5,0,3,0.149533,0.164286,0.313808,0.153226,4
3,38,0.080000,0.157128,2,5,0.166667,Male,3,0.134615,0.124814,...,13,5,8,7,5,0.134183,0.114119,0.142857,0.160000,3
4,32,0.149569,0.157128,10,1,0.161638,Male,1,0.168712,0.255319,...,9,2,6,0,4,0.134183,0.229070,0.142857,0.153226,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,42,0.149569,0.157128,5,4,0.161638,Female,1,0.181507,0.255319,...,10,5,3,0,2,0.134183,0.229070,0.142857,0.153226,3
4406,29,0.149569,0.157128,2,4,0.161638,Male,1,0.162162,0.100917,...,10,2,3,0,2,0.134183,0.114119,0.142857,0.160000,3
4407,25,0.149569,0.157128,25,2,0.166667,Male,2,0.168712,0.124814,...,5,4,4,1,2,0.252071,0.165533,0.142857,0.153226,4
4408,42,0.149569,0.150224,18,2,0.161638,Male,1,0.162162,0.100917,...,10,2,9,7,8,0.134183,0.229070,0.142857,0.160000,3


In [141]:
onehot.transform(res_target[cat_no_diff]).toarray()

array([[0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [142]:
res_onehot =pd.DataFrame(onehot.transform(res_target[cat_no_diff]).toarray(), columns = onehot.get_feature_names_out())

In [143]:
res_encoded = pd.concat([res_target.drop(columns = cat_no_diff), res_onehot], axis = 1)

In [144]:
res_scaled = pd.DataFrame(scaler.transform(res_encoded.drop(columns = res_onehot.columns)), columns = scaler.get_feature_names_out())
res_scaled = pd.concat([res_scaled, res_onehot], axis = 1)

In [145]:
model.predict(res_scaled)

array([0, 1, 0, ..., 0, 0, 0])